<a href="https://colab.research.google.com/github/santiagoprado12/Tesis/blob/main/pre_procesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas
import geopandas
import datetime 
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objects as go

     |████████████████████████████████| 1.0MB 5.1MB/s 
     |████████████████████████████████| 14.8MB 299kB/s 
     |████████████████████████████████| 6.5MB 43.5MB/s 


In [ ]:
# make sure to install these packages before running:
#!pip install pandas
!pip install sodapy
##
##
import pandas as pd
from sodapy import Socrata
 
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.datos.gov.co", None)
 
# Example authenticated client (needed for non-public datasets):
# client = Socrata(www.datos.gov.co,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")
 
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("gt2j-8ykr",limit=2217001) 
 
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

################################################################################

url = "https://opendata.arcgis.com/datasets/782122624f364fbdbd7e287b96c4a358_6.geojson"
df = geopandas.read_file(url)

In [ ]:
results_df["estado"].unique()

array(['Leve', 'Fallecido', 'N/A', 'leve', 'Moderado', 'Grave',
       'moderado', 'LEVE'], dtype=object)

In [ ]:
diagnosticados=results_df[pd.notnull(results_df["fecha_de_notificaci_n"])]["fecha_de_notificaci_n"]
diagnosticados_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in diagnosticados]#Creando una nueva columna formato datetime
diagnosticados_sint= results_df[pd.notnull(results_df["fecha_de_notificaci_n"])]["fecha_inicio_sintomas"]



df_diagnosticados_date_fmat=pd.DataFrame({'diagnosticadosN': diagnosticados})
df_diagnosticados_date_fmat['diagnosticadosS']=diagnosticados_sint

df_diagnosticados_date_fmat['diagnosticadosS'].fillna(df_diagnosticados_date_fmat['diagnosticadosN'], inplace=True)

diagnosticados_sint_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in df_diagnosticados_date_fmat['diagnosticadosS']]#Creando una nueva columna formato datetime


diagnostico=[]
for i,d in enumerate(diagnosticados_date_fmat):
  if d<diagnosticados_sint_date_fmat[i]:
    diagnostico.append(d)
  else:
    diagnostico.append(diagnosticados_sint_date_fmat[i])



df_diagnosticados_date_fmat=pd.DataFrame({'diagnosticados': diagnostico})

In [ ]:
diagnosticados1=results_df[pd.notnull(results_df["fecha_de_notificaci_n"])]["fecha_de_notificaci_n"]
diagnosticados_date_fmat1=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in diagnosticados1]#Creando una nueva columna formato datetime
diagnosticados_sint1= results_df[pd.notnull(results_df["fecha_de_notificaci_n"])]["fecha_inicio_sintomas"]



df_diagnosticados_date_fmat1=pd.DataFrame({'diagnosticadosN': diagnosticados})
df_diagnosticados_date_fmat1['diagnosticadosS']=diagnosticados_sint1

df_diagnosticados_date_fmat1['diagnosticadosS'].fillna(df_diagnosticados_date_fmat1['diagnosticadosN'], inplace=True)

diagnosticados_sint_date_fmat1=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in df_diagnosticados_date_fmat1['diagnosticadosS']]#Creando una nueva columna formato datetime


diagnostico=[]
for i,d in enumerate(diagnosticados_date_fmat1):

  if d<diagnosticados_sint_date_fmat1[i]:
    diagnostico.append(d)
  else:
    diagnostico.append(diagnosticados_sint_date_fmat1[i]-datetime.timedelta(days=8))


df_expuestos_date_fmat=pd.DataFrame({'expuestos': diagnostico})

asintomaticos_rec=results_df[(pd.isnull(results_df["fecha_inicio_sintomas"])) & (results_df["recuperado"]=="Recuperado")]["fecha_recuperado"]
asintomaticos_rec_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in asintomaticos_rec]#Creando una nueva columna formato datetime
df_asintomaticos_rec_date_fmat=pd.DataFrame({'asintomaticos_rec': asintomaticos_rec_date_fmat})


asintomaticos_muer=results_df[(pd.isnull(results_df["fecha_inicio_sintomas"])) & ((results_df["recuperado"]=='fallecido') | (results_df["recuperado"]=='Fallecido'))]["fecha_muerte"]
asintomaticos_muer_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in asintomaticos_muer]#Creando una nueva columna formato datetime
df_asintomaticos_muer_date_fmat=pd.DataFrame({'asintomaticos_muer': asintomaticos_muer_date_fmat})

In [ ]:
muertes=results_df[(pd.notnull(results_df["fecha_muerte"])) & ((results_df["recuperado"]=='fallecido') | (results_df["recuperado"]=='Fallecido'))]["fecha_muerte"]
muertes_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in muertes]#Creando una nueva columna formato datetime
df_muertes_date_fmat=pd.DataFrame({'muertes': muertes_date_fmat})

In [ ]:
#recuperados=results_df[(pd.notnull(results_df["fecha_recuperado"]))]["fecha_recuperado"]
recuperados=results_df[results_df["recuperado"]=="Recuperado"]["fecha_recuperado"]
recuperados_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in recuperados]#Creando una nueva columna formato datetime
df_recuperados_date_fmat=pd.DataFrame({'recuperados': recuperados_date_fmat})

In [ ]:
sintomaticos=results_df[pd.notnull(results_df["fecha_inicio_sintomas"]) ]["fecha_inicio_sintomas"]
sintomaticos_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in sintomaticos]#Creando una nueva columna formato datetime
df_sintomaticos_date_fmat=pd.DataFrame({'sintomaticos': sintomaticos_date_fmat})


sintomaticos_rec=results_df[(pd.notnull(results_df["fecha_inicio_sintomas"])) & (results_df["recuperado"]=="Recuperado")]["fecha_recuperado"]
sintomaticos_rec_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in sintomaticos_rec]#Creando una nueva columna formato datetime
df_sintomaticos_rec_date_fmat=pd.DataFrame({'sintomaticos_rec': sintomaticos_rec_date_fmat})


sintomaticos_muer=results_df[(pd.notnull(results_df["fecha_inicio_sintomas"])) & ((results_df["recuperado"]=='fallecido') | (results_df["recuperado"]=='Fallecido'))]["fecha_muerte"]
sintomaticos_muer_date_fmat=[datetime.datetime.strptime(str(x),"%d/%m/%Y %H:%M:%S") for x in sintomaticos_muer]#Creando una nueva columna formato datetime
df_sintomaticos_muer_date_fmat=pd.DataFrame({'sintomaticos_muer': sintomaticos_muer_date_fmat})

In [ ]:
fecha_mayor=str(results_df["fecha_de_notificaci_n"].unique()[-1])
fecha_mayor1=datetime.datetime.strptime(str(fecha_mayor),"%d/%m/%Y %H:%M:%S")

In [ ]:
c=str(results_df["fecha_reporte_web"][0])
c1=datetime.datetime.strptime(c,"%d/%m/%Y %H:%M:%S")-datetime.timedelta(days=8)
c2=[c1]
i=1
while c2[-1] != fecha_mayor1:
    c2.append(c2[0]+datetime.timedelta(days=i))
    i+=1

In [ ]:
casos_diarios={}
casos_diarios["fechas"]=c2
casos_diarios_DF=pd.DataFrame(data=casos_diarios)

In [ ]:
fecha_Actu=[f.split('+')[0] for f in df["FECHA_ACTUALIZACION"]]
FECHA_ACTUALIZACION_DATE_FMAT=[datetime.datetime.strptime(str(x),"%Y-%m-%dT%H:%M:%S") for x in fecha_Actu]#Creando una nueva columna formato datetime
df.tail()

,OBJECTID,NUEVOS_CASOS,TOTAL_CASOS,TOTAL_MUERTES,TOTAL_RECUPERADOS,FECHA_ACTUALIZACION,GlobalID,NUEVOS_MUERTOS,NUEVO_RECUPERADOS,geometry
358,910,3343,2248135,59660,2145450,2021-02-27T05:00:00+00:00,b0f3ab3a-b3e4-4e6c-b582-3fa328af7c89,142,3576,None
359,911,3555,2251690,59766,2148249,2021-02-28T05:00:00+00:00,364b3615-1942-4278-bc7c-4a81ec85c2b7,106,2799,None
360,912,3570,2255260,59866,2151633,2021-03-01T05:00:00+00:00,31105e39-f019-4577-92c1-0b7ff1032bfb,100,3384,None
361,913,4339,2259599,59972,2156057,2021-03-02T05:00:00+00:00,4975f47a-b9bf-44a4-8670-a742943811b8,106,4424,None
362,914,3047,2262646,60082,2160555,2021-03-03T05:00:00+00:00,91e8b974-38dc-4f39-a2dc-4caac97d51ca,110,4498,None


In [ ]:
casos_diarios_DF["casos por dia"]=list(map(lambda f: sum(df_diagnosticados_date_fmat['diagnosticados']==f), casos_diarios_DF["fechas"]))

In [ ]:
casos_diarios_DF["fallecidos por dia"]=list(map(lambda f: sum(df_muertes_date_fmat['muertes']==f), casos_diarios_DF["fechas"]))

In [ ]:
casos_diarios_DF["recuperados por dia"]=list(map(lambda f: sum(df_recuperados_date_fmat['recuperados']==f), casos_diarios_DF["fechas"]))

In [ ]:
casos_diarios_DF["sintomaticos activos"]=list(map(lambda f: (sum(df_sintomaticos_date_fmat['sintomaticos']==f))-(sum(df_sintomaticos_rec_date_fmat['sintomaticos_rec']==f))-(sum(df_sintomaticos_muer_date_fmat['sintomaticos_muer']==f)), casos_diarios_DF["fechas"]))

In [ ]:
casos_diarios_DF["expuestos"]=list(map(lambda f: sum(df_expuestos_date_fmat['expuestos']==f)-(sum(df_sintomaticos_date_fmat['sintomaticos']==f))-(sum(df_asintomaticos_rec_date_fmat['asintomaticos_rec']==f))-(sum(df_asintomaticos_muer_date_fmat['asintomaticos_muer']==f)), casos_diarios_DF["fechas"]))

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["casos por dia"],
    name="Preprocesamiento propio"
))


fig.add_trace(go.Scatter(
    x=FECHA_ACTUALIZACION_DATE_FMAT,
    y=df["NUEVOS_CASOS"],
    name="base de datos"
))

fig.update_layout(title="Casos nuevos por dia")

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["fallecidos por dia"],
    name="Preprocesamiento propio"
))


fig.add_trace(go.Scatter(
    x=FECHA_ACTUALIZACION_DATE_FMAT,
    y=df["NUEVOS_MUERTOS"],
    name="base de datos"
))

fig.update_layout(title="Muertos por dia")

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["recuperados por dia"],
    name="Preprocesamiento propio"
))


fig.add_trace(go.Scatter(
    x=FECHA_ACTUALIZACION_DATE_FMAT,
    y=df["NUEVO_RECUPERADOS"],
    name="base de datos"
))
fig.update_layout(title="Recuperados por dia")

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
len(results_df[(results_df["recuperado"]=="Recuperado") & (results_df["tipo_recuperacion"]=="Tiempo")]["fecha_recuperado"])/len(results_df[results_df["recuperado"]=="Recuperado"]["fecha_recuperado"])

0.8818085335977465

In [ ]:
sum(df["NUEVOS_CASOS"])

2262563

In [ ]:
sum(casos_diarios_DF["casos por dia"])

2217001

In [ ]:
sum(casos_diarios_DF["fallecidos por dia"])

58780

In [ ]:
sum(df["NUEVOS_MUERTOS"])

60081

In [ ]:
sum(df["NUEVO_RECUPERADOS"])

2160525

In [ ]:
sum(casos_diarios_DF["recuperados por dia"])

2102774

In [ ]:
casos_diarios_DF["acumulado muertes"] = [sum(casos_diarios_DF["fallecidos por dia"][:x]) for x in range(0,len(casos_diarios_DF["fallecidos por dia"]))]
casos_diarios_DF["acumulado recuperados"] = [sum(casos_diarios_DF["recuperados por dia"][:x]) for x in range(0,len(casos_diarios_DF["recuperados por dia"]))]
casos_diarios_DF["acumulado casos"] = [sum(casos_diarios_DF["casos por dia"][:x]) for x in range(0,len(casos_diarios_DF["casos por dia"]))]
casos_diarios_DF["acumulado sintomaticos"] = [sum(casos_diarios_DF["sintomaticos activos"][:x]) for x in range(0,len(casos_diarios_DF["sintomaticos activos"]))]
casos_diarios_DF["acumulado expuestos"] = [sum(casos_diarios_DF["expuestos"][:x]) for x in range(0,len(casos_diarios_DF["expuestos"]))]

In [ ]:
casos_diarios_DF["activos por dia"] = casos_diarios_DF["acumulado casos"]-casos_diarios_DF["acumulado recuperados"]-casos_diarios_DF["acumulado muertes"]

In [ ]:
casos_diarios_DF['asintomaticos']=casos_diarios_DF["activos por dia"]-casos_diarios_DF["acumulado sintomaticos"]

In [ ]:
actv=df["NUEVOS_CASOS"]-df["NUEVOS_MUERTOS"]-df["NUEVO_RECUPERADOS"]
actv

0         1
1         0
2         0
3         2
4         0
       ... 
358    -375
359     650
360      86
361    -191
362   -1561
Length: 363, dtype: int64

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["activos por dia"],
    name="Preprocesamiento propio"
))

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["acumulado sintomaticos"],
    name="sintomaticos activos"
))

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["acumulado expuestos"],
    name="expuestos"
))
fig.update_layout(title="Activos por dia")

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["acumulado muertes"],
    name="fallecidos Preprocesamiento propio"
))

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["acumulado recuperados"],
    name="recuperados Preprocesamiento propio"
))

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["acumulado casos"],
    name="infectados Preprocesamiento propio"
))


fig.add_trace(go.Scatter(
    x=FECHA_ACTUALIZACION_DATE_FMAT,
    y=df["TOTAL_RECUPERADOS"],
    name="recuperados base de datos"
))

fig.add_trace(go.Scatter(
    x=FECHA_ACTUALIZACION_DATE_FMAT,
    y=df["TOTAL_CASOS"],
    name="infectados base de datos"
))

fig.add_trace(go.Scatter(
    x=FECHA_ACTUALIZACION_DATE_FMAT,
    y=df["TOTAL_MUERTES"],
    name="muertos base de datos"
))

fig.update_layout(title="Recuperados por dia")

fig.update_layout(showlegend=True)

fig.show()

In [ ]:
casos_diarios_DF[:340]

,fechas,casos por dia,fallecidos por dia,recuperados por dia,sintomaticos activos,expuestos,acumulado muertes,acumulado recuperados,acumulado casos,acumulado sintomaticos,acumulado expuestos,activos por dia,asintomaticos
0,2020-02-27,1,0,0,1,14,0,0,0,0,0,0,0
1,2020-02-28,1,0,0,1,13,0,0,1,1,14,1,0
2,2020-02-29,2,0,0,2,9,0,0,2,2,27,2,0
3,2020-03-01,5,0,0,5,17,0,0,4,4,36,4,0
4,2020-03-02,4,0,0,4,33,0,0,9,9,53,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,2021-01-27,6797,333,16583,-10051,-3277,53401,1847091,2146621,221163,66298,246129,24966
336,2021-01-28,6440,316,16404,-11297,-2006,53734,1863674,2153418,211112,63021,236010,24898
337,2021-01-29,5805,316,16439,-11863,-1945,54050,1880078,2159858,199815,61015,225730,25915
338,2021-01-30,4702,296,14922,-9156,-3812,54366,1896517,2165663,187952,59070,214780,26828


In [ ]:
casos_diarios_DF[:340].to_csv(r'Casos diarios.csv', index = False)